In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

### Load data, set initial configuration

In [2]:
all_in_one = pd.read_csv('../data/all_in_one.csv')

In [3]:
all_in_one_no_id = all_in_one.drop(columns=['Unnamed: 0'])
all_in_one_no_id = all_in_one_no_id.replace([np.inf, -np.inf], np.nan)
all_countries = all_in_one_no_id['Country.Name'].unique()
all_variables = all_in_one_no_id.columns[4:]

In [4]:
# Prepare a color palette for all unique countries in the dataset
sns.set_theme(style="dark")
palette = dict(zip(all_countries, sns.color_palette("husl", len(all_countries))))

### Create panel data set for each outcome variable

In [5]:
from create_panel_dataset import remove_cols_with_few_observations, create_outcome_summaries, remove_outliers_iqr

Start with year 1995 where ecirank data is available, end with 2015, the last year for which energy data is available. 
Only keep observations where energy use data is available.
Remove selected columns (selection process?)

In [6]:
all_in_one_filtered = all_in_one_no_id.loc[all_in_one_no_id['Year'].between(1995, 2015)]
all_in_one_filtered_with_energy_only = all_in_one_filtered[all_in_one_filtered[['energy']].notnull().all(1)]
all_in_one_no_id_selected_cols = all_in_one_filtered_with_energy_only.drop(
    columns=['netmigration', 'wealth', 'patents', 'concentration', 'selfemployed', 'grosscapital', 'gdppercap',
             'gnipercap'])

Create separate data frames containing observations with data available for each outcome variable. 

After subsetting is done, create final data frames and utility variables. 

Remove columns with few observations.

In [7]:
outcome_variables = ['lifeexpectancy_over_energy', 'nutrition_over_energy', 'education_over_energy',
                     'sanitation_over_energy']
summaries_panel = create_outcome_summaries(
    df=all_in_one_no_id_selected_cols,
    outcome_variables=outcome_variables,
    index_offset=3,
    outcome_index_offset=17,
    remove_cols_func=remove_cols_with_few_observations,
    remove_outliers_func=remove_outliers_iqr,
)
summaries_cross_sectional = create_outcome_summaries(
    df=all_in_one_no_id_selected_cols,
    outcome_variables=outcome_variables,
    filter_year=2012,
    index_offset=3,
    outcome_index_offset=17,
    remove_cols_func=remove_cols_with_few_observations,
    remove_outliers_func=remove_outliers_iqr,
)

/Users/supernaturval/Documents/GitHub/provisioning-pipeline/notebooks/create_panel_dataset.py:137: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  Q1 = df.quantile(0.25)
/Users/supernaturval/Documents/GitHub/provisioning-pipeline/notebooks/create_panel_dataset.py:138: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  Q3 = df.quantile(0.75)
/Users/supernaturval/Documents/GitHub/provisioning-pipeline/notebooks/create_panel_dataset.py:146: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version. Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == r

In [8]:
for summary in summaries_panel:
    print(summary[list(summary.keys())[0]].info())
    print(summary[list(summary.keys())[1]].info())
    print(summary[list(summary.keys())[2]].info())
    print(summary[list(summary.keys())[3]])
    print(summary[list(summary.keys())[4]])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1087 entries, 72 to 11579
Data columns (total 53 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Year                          1087 non-null   int64  
 1   Country.Code                  1087 non-null   object 
 2   Country.Name                  1087 non-null   object 
 3   agriculture                   1071 non-null   float64
 4   cleanfuel                     831 non-null    float64
 5   corruption                    890 non-null    float64
 6   electricity                   1050 non-null   float64
 7   exports                       1083 non-null   float64
 8   FDI                           1050 non-null   float64
 9   fossil                        1085 non-null   float64
 10  gini                          554 non-null    float64
 11  goveffectiveness              890 non-null    float64
 12  govexp                        1077 non-null   float64
 13  i

## Exploratory analysis

In [9]:
from exploratory_analysis import create_summary_df, plot_summary_variable, plot_outliers, plot_correlation_matrix, \
    plot_histograms

In [10]:
# plot_summary_variable(all_lifeexp_over_energy_summary_df, 'observations')
# plot_summary_variable(all_lifeexp_over_energy_summary_df, 'num_countries')
# plot_summary_variable(all_lifeexp_over_energy_summary_df, 'num_years')

## Cross-validation: Best subset, Lasso, PCA

In [11]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.impute import KNNImputer
# from sklearn.feature_selection import SelectFromModel
# from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from abess import LinearRegression
# from abess import SparsePCA
import numpy as np

In [12]:
def panel_cross_validation(models, model_names, x, y):
    """
    Conducts cross-validation using different models and returns the best model along with details on evaluated models.
    
    Parameters:
    - models: list of machine learning models to evaluate
    - model_names: list of names for the models
    - x: feature matrix
    - y: target vector
    
    Returns:
    - best_model: the best model
    - results: a dictionary containing the results of all evaluated models
    """
    candidate_splits = [5]
    # do this initially [2, 3, 5, 7, 10, 15]
    results = {}

    best_model = None
    best_score = float('inf')
    best_model_name = ''
    best_n_splits = 0
    best_lasso_param = None

    for model, model_name in zip(models, model_names):
        for n in candidate_splits:
            tscv = TimeSeriesSplit(n_splits=n)
            lasso_param = None
            coefs = None
            intercept = None

            if model_name == 'Lasso':
                parameters = {'alpha': [0.01, 0.1, 0.5, 0.59, 0.6, 0.7, 1, 10, 100]}
                grid = GridSearchCV(model, parameters, scoring='neg_mean_squared_error', cv=tscv)
                pipeline = Pipeline(
                    [('imputer', KNNImputer(n_neighbors=5)), ('scaler', StandardScaler()), ('model', grid)])
                pipeline.fit(x, y)
                lasso_param = grid.best_params_
                best_lasso_model = grid.best_estimator_
                coefs = best_lasso_model.coef_
                intercept = best_lasso_model.intercept_

            elif model_name == 'PCA':
                pipeline = Pipeline(
                    [('imputer', KNNImputer(n_neighbors=5)), ('scaler', StandardScaler()), ('pca', model),
                     ('regression', LinearRegression())])
            else:
                pipeline = Pipeline(
                    [('imputer', KNNImputer(n_neighbors=5)), ('scaler', StandardScaler()), ('model', model)])

            pipeline.fit(x, y)

            if model_name != 'Lasso' and model_name != 'PCA':
                fitted_model = pipeline.named_steps['model']
                coefs = fitted_model.coef_
                intercept = fitted_model.intercept_

            scores = cross_val_score(pipeline, x, y, cv=tscv, scoring='neg_mean_squared_error')
            mean_score = -np.mean(scores)
            std_score = np.std(scores)

            if mean_score < best_score:
                best_score = mean_score
                best_model = pipeline
                best_model_name = model_name
                best_n_splits = n
                best_lasso_param = lasso_param

            print(
                f"Model: {model_name}, Mean MSE: {mean_score:.4f}, Std MSE: {std_score:.4f}, n_splits: {n}, Lasso alpha: {lasso_param}")
            results[model_name + '_' + str(n)] = {'Mean_MSE': mean_score, 'Std_MSE': std_score,
                                                  'Lasso_alpha': lasso_param, 'Coefficients': coefs,
                                                  'Intercept': intercept}

    print(
        f"Best Model: {best_model_name}, Mean MSE: {best_score}, n_splits: {best_n_splits}, Lasso alpha: {best_lasso_param}")
    return best_model, results


In [13]:
# Initialize the models
best_subset_model = LinearRegression()
lasso_model = Lasso(max_iter=10000)
pca_model = PCA()

# Combine these into a list
models = [best_subset_model, lasso_model, pca_model]
model_names = ['Best Subset', 'Lasso', 'PCA']

In [14]:
lifeexp_model, lifeexp_cv_results = panel_cross_validation(models, model_names,
                                                           indicators_lifeexp_over_energy_with_energy_no_outliers,
                                                           outcome_lifeexp_over_energy)

NameError: name 'indicators_lifeexp_over_energy_with_energy_no_outliers' is not defined

### Anaylzing cross-validation results

In [ ]:
def plot_cross_validation_results(results):
    model_names = []
    mean_mses = []
    std_mses = []
    coefficients = []

    for model_name, model_result in results.items():
        model_names.append(model_name)
        mean_mses.append(model_result['Mean_MSE'])
        std_mses.append(model_result['Std_MSE'])
        coefficients.append(model_result['Coefficients'])

    fig, axs = plt.subplots(2, 1, figsize=(12, 12))

    # Plotting Mean MSE with error bars for Std MSE
    axs[0].barh(model_names, mean_mses, xerr=std_mses, align='center', alpha=0.7, color='b', ecolor='black', capsize=10)
    axs[0].set_xlabel('Mean MSE')
    axs[0].set_title('Model Performance')

    # Plotting coefficients for the models
    for idx, coef in enumerate(coefficients):
        axs[1].plot(coef, label=f'{model_names[idx]} coefficients')

    axs[1].set_xlabel('Feature index')
    axs[1].set_ylabel('Coefficient value')
    axs[1].set_title('Model Coefficients')
    axs[1].legend()

    plt.tight_layout()
    plt.show()

In [ ]:
def filter_and_find_best_model(results):
    best_model = None
    best_score = float('inf')

    for model_key, model_info in results.items():
        model_name = model_key
        model_coef = model_info.get('Coefficients', None)
        mean_score = model_info.get('Mean_MSE', float('inf'))

        # print(f"Debug: model_coef = {model_coef}")

        if model_coef is not None:
            if not all(coef == 0 for coef in model_coef):
                if mean_score < best_score:
                    best_score = mean_score
                    best_model = model_name  # Changed this to model_name, as we are not storing the actual model object in this example

    return best_model


In [ ]:
# plot_model_results(lifeexp_cv_results)
# best_model_no_zero_coef = filter_and_find_best_model(lifeexp_cv_results)
# print(best_model_no_zero_coef)

### Best model?
Lasso with alpha 0.59 and n_splits 5. 
! Lasso with alpha 1 has better MSE but all coefficients are 0.

Try removing "selfemployed" column because it's collinear with "wageworker" and "concentration" as an outlier

In [ ]:
lifeexp_lasso = Pipeline([('imputer', KNNImputer(n_neighbors=5)), ('scaler', StandardScaler()),
                          ('model', Lasso(max_iter=10000, alpha=0.59))])

lifeexp_lasso.fit(indicators_lifeexp_over_energy_with_energy_no_outliers, outcome_lifeexp_over_energy)
lifeexp_lasso_coef = lifeexp_lasso.named_steps['model'].coef_

important_features = []
for i, coef in enumerate(lifeexp_lasso_coef):
    if coef != 0:
        important_features.append((i, coef))

print("Important features:", important_features)
print(lifeexp_lasso.named_steps['model'].intercept_)
indicators_lifeexp_over_energy_important = indicators_lifeexp_over_energy_with_energy_no_outliers.iloc[:,
                                           [i for i, coef in important_features]]
indicators_lifeexp_over_energy_important_2012 = indicators_lifeexp_over_energy_with_energy_2012_no_outliers.iloc[:,
                                                [i for i, coef in important_features]]
print(len(indicators_lifeexp_over_energy_important.columns), len(indicators_lifeexp_over_energy.columns))

## Panel regression with cross-validation

In [ ]:
# simple linear regression
from sklearn.linear_model import LinearRegression

lifeexp_lasso_linear = Pipeline([('imputer', KNNImputer(n_neighbors=5)), ('model', LinearRegression())])
lifeexp_lasso_linear.fit(indicators_lifeexp_over_energy_important, outcome_lifeexp_over_energy)
lifeexp_lasso_linear_coef = lifeexp_lasso_linear.named_steps['model'].coef_
print("Linear regression coefficients:", lifeexp_lasso_linear_coef)
print("Adjusted r squared",
      lifeexp_lasso_linear.score(indicators_lifeexp_over_energy_important, outcome_lifeexp_over_energy))

In [ ]:
# try the same regression only for year 2012
lifeexp_lasso_linear_2012 = Pipeline([('imputer', KNNImputer(n_neighbors=5)), ('model', LinearRegression())])
lifeexp_lasso_linear_2012.fit(indicators_lifeexp_over_energy_important_2012, outcome_lifeexp_over_energy_2012)
lifeexp_lasso_linear_coef_2012 = lifeexp_lasso_linear_2012.named_steps['model'].coef_
print("Linear regression coefficients:", lifeexp_lasso_linear_coef_2012)
print("Adjusted r squared",
      lifeexp_lasso_linear_2012.score(indicators_lifeexp_over_energy_important_2012, outcome_lifeexp_over_energy_2012))

In [ ]:
# Extract the feature names from your DataFrame (change this line if your feature names are stored differently)
indicators_lifeexp_over_energy_names = indicators_lifeexp_over_energy_important.columns

# Create a list of (coefficient, feature_name) tuples and sort them based on the coefficients
indicators_lifeexp_over_energy_names_sorted = sorted(
    zip(lifeexp_lasso_linear_coef, indicators_lifeexp_over_energy_names))

# Separate the tuples into two lists
lifeexp_lasso_coef_sorted, lifeexp_over_energy_names_sorted = zip(*indicators_lifeexp_over_energy_names_sorted)

# Create a bar plot
plt.figure(figsize=(10, 8))
plt.barh(range(len(lifeexp_lasso_coef_sorted)), lifeexp_lasso_coef_sorted, align='center')
plt.yticks(range(len(lifeexp_lasso_coef_sorted)), lifeexp_over_energy_names_sorted)
plt.xlabel('Coefficient Value')
plt.ylabel('Feature Names')
plt.title('Feature Importances')
plt.show()

### Plotting best subset results

In [ ]:
# def plot_best_subset_results(model_coef, predictor_df):
#     """
#     Plots non-zero coefficients of the model against corresponding variable names.

#     Parameters:
#         model_coef (array-like): Coefficients of the model.
#         predictor_df (pd.DataFrame): DataFrame containing the predictor variables.

#     Returns:
#         None
#     """
#     # Extract column names and coefficients from the DataFrame
#     column_names = predictor_df.columns

#     # Find non-zero coefficients
#     non_zero_indices = np.nonzero(model_coef)[0]

#     # Extract non-zero coefficients and corresponding labels
#     non_zero_coef = model_coef[non_zero_indices]
#     non_zero_labels = column_names[non_zero_indices]

#     # Create a DataFrame to sort by absolute coefficient values
#     df = pd.DataFrame({'Variable': non_zero_labels, 'Coefficient': non_zero_coef})
#     df = df.sort_values(by='Coefficient', key=abs, ascending=True)

#     # Plotting
#     plt.figure(figsize=(10, 8))
#     plt.barh(df['Variable'], df['Coefficient'], color='skyblue')
#     plt.xlabel('Coefficient Value')
#     plt.ylabel('Variables')
#     plt.title('Non-zero Coefficients')
#     plt.grid(True, linestyle='--', alpha=0.7)

#     # Add text annotations to the bars
#     for index, value in enumerate(df['Coefficient']):
#         plt.text(value, index, f'{value:.4f}')

#     plt.show()

# Example usage:
# plot_best_subset_results(model.coef_, indicators_lifeexp_over_energy_imputed_df)